<a href="https://colab.research.google.com/github/zihadbappy/bnfake/blob/main/BanFakeNews_CNN_with_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!git clone https://github.com/zihadbappy/FakeDatasets.git
import pandas as pd
Dataset= pd.read_csv('/content/FakeDatasets/banfakenews_no_duplicate.csv')

fatal: destination path 'FakeDatasets' already exists and is not an empty directory.


In [28]:

Dataset.head()

,headline,content,label
0,মাঝ আকাশে পর্ন দেখার অভিযোগে পাইলট আটক,বিমান আকাশে খেয়াল শিশু পর্নোগ্রাফির অ্যাপ ডাউ...,0
1,শান্তিতে নোবেল পুরস্কার পাচ্ছেন শেখ হাসিনা,মিয়ানমারের রাখাইন বাংলাদেশে ছুটে আসা রোহিঙ্গা ...,0
2,কানাবগির ছা স্পোর্টিং ক্লাব ও রামগরুড়ের ছানা স...,তালগাছপুরের কানা বগির ছা স্পোর্টিং ক্লাব বার্ষ...,0
3,আজ রাতে পৃথিবীতে ধেয়ে আসছে ক্ষতিকর মহাজাগতিক র...,রাতে পৃথিবীতে ধেয়ে আসছে ক্ষতিকর উচ্চ তেজস্ক্রি...,0
4,গালিগালাজ স্বাস্থ্যের জন্য ভালো,স্কুলকলেজ বাসাবাড়ি অফিসেই অতি মানসিক চাপে গাল...,0


In [29]:
df = pd.DataFrame(Dataset)

In [30]:
df = df.reset_index(drop=True)
# print(df)

In [31]:
df=df.dropna()
df

,headline,content,label
0,মাঝ আকাশে পর্ন দেখার অভিযোগে পাইলট আটক,বিমান আকাশে খেয়াল শিশু পর্নোগ্রাফির অ্যাপ ডাউ...,0
1,শান্তিতে নোবেল পুরস্কার পাচ্ছেন শেখ হাসিনা,মিয়ানমারের রাখাইন বাংলাদেশে ছুটে আসা রোহিঙ্গা ...,0
2,কানাবগির ছা স্পোর্টিং ক্লাব ও রামগরুড়ের ছানা স...,তালগাছপুরের কানা বগির ছা স্পোর্টিং ক্লাব বার্ষ...,0
3,আজ রাতে পৃথিবীতে ধেয়ে আসছে ক্ষতিকর মহাজাগতিক র...,রাতে পৃথিবীতে ধেয়ে আসছে ক্ষতিকর উচ্চ তেজস্ক্রি...,0
4,গালিগালাজ স্বাস্থ্যের জন্য ভালো,স্কুলকলেজ বাসাবাড়ি অফিসেই অতি মানসিক চাপে গাল...,0
...,...,...,...
2347,হুসাইন রা ও কারবালা ট্র্যাজেডি,রাসুলুল্লাহ সাএর অতি আদরের নাতি হুসাইন রা খাতু...,1
2348,বামজোটের ওপর হামলা বিএনপির প্রতিবাদ,বাম গণতান্ত্রিক জোটের নির্বাচন কমিশন অভিমুখে ক...,1
2349,আবারও ব্যর্থ ব্যাটসম্যানরা,ভারতের বিপক্ষে রানে উইকেট হারিয়ে কার্যত ম্যাচ ...,1
2350,অংশগ্রহণমূলক নির্বাচন এবং বিদেশি শক্তির হাত,সংবিধান অনুযায়ী একাদশ জাতীয় সংসদ নির্বাচন ডিসে...,1


In [32]:
df.groupby('label').agg('count')

,headline,content
label,,
0,1176,1176
1,1175,1175


# CNN

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [34]:
X = df.content
Y = df.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [35]:
print(Y)

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [36]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25, random_state=99)

In [37]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [38]:
  # calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

In [39]:
# calculate vocabulary size
vocab_size = 86000

In [40]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

In [41]:

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping 

In [42]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [43]:
tokenizer = create_tokenizer(X_train)
# calculate max document length
length = max_length(X_train)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, X_train, length)
print(trainX.shape)
 
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], array(Y_train), epochs=16, batch_size=32,
    callbacks=[EarlyStopping(monitor="accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])
# save the model
model.save('model.h5')

Max document length: 1861
Vocabulary size: 53330
(1763, 1861)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 1861)]       0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 1861)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 1861)]       0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 1861, 100)    5333000     ['input_4[0][0]']                
                              

In [44]:
type(Y_test)

numpy.ndarray

In [45]:
loss, acc = model.evaluate([trainX,trainX,trainX], array(Y_train), verbose=0)
print('Train Accuracy: %f' % (acc*100))

testX = encode_text(tokenizer, X_test, length)

loss, acc = model.evaluate([testX,testX,testX],array(Y_test), verbose=0)
print('Test Accuracy: %f' % (acc*100))

Train Accuracy: 100.000000
Test Accuracy: 92.176872


In [46]:
testX

array([[ 6799,  1317,  2236, ...,     0,     0,     0],
       [ 6208,   121,   246, ...,     0,     0,     0],
       [   26,   929,  9487, ...,     0,     0,     0],
       ...,
       [   66,   831,  6771, ...,     0,     0,     0],
       [  120,   791,     3, ...,     0,     0,     0],
       [  525,  2018, 10264, ...,     0,     0,     0]], dtype=int32)

In [47]:
trainX

array([[   41,   717,  1133, ...,     0,     0,     0],
       [   20,  1024, 10986, ...,     0,     0,     0],
       [ 4416,   514,  1306, ...,     0,     0,     0],
       ...,
       [  275,    49,   593, ...,     0,     0,     0],
       [ 9687, 15773, 24683, ...,     0,     0,     0],
       [  432,   811,   268, ...,     0,     0,     0]], dtype=int32)

In [48]:
CNN_pred_y = model.predict([testX, testX, testX])

In [49]:
from sklearn.metrics import classification_report

In [50]:
classification_report(Y_test, np.around(CNN_pred_y), output_dict=True)

{'0': {'precision': 0.9530201342281879,
  'recall': 0.8987341772151899,
  'f1-score': 0.9250814332247557,
  'support': 316},
 '1': {'precision': 0.8896551724137931,
  'recall': 0.9485294117647058,
  'f1-score': 0.9181494661921707,
  'support': 272},
 'accuracy': 0.9217687074829932,
 'macro avg': {'precision': 0.9213376533209905,
  'recall': 0.9236317944899479,
  'f1-score': 0.9216154497084632,
  'support': 588},
 'weighted avg': {'precision': 0.9237084512120052,
  'recall': 0.9217687074829932,
  'f1-score': 0.9218748090192062,
  'support': 588}}

In [51]:
np.around(CNN_pred_y[20:25])

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [52]:
Y_test[20:25]

array([[0],
       [0],
       [1],
       [1],
       [1]])